Benchmarking Hardware

In [1]:
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)

/opt/conda/envs/py39_llama_hf/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(torch.__version__)
print(transformers.__version__)

2.0.1+cu118
4.31.0


In [3]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)
print("done creating pipeline")


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/envs/py39_llama_hf/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda113.so
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /opt/conda/envs/py39_llama_hf/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda113.so...


/opt/conda/envs/py39_llama_hf/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /opt/conda did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
Loading checkpoint shards: 100%|██████████| 2/2 [01:19<00:00, 39.91s/it]
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


done creating pipeline


In [5]:
import time

benchmark_results = [] # (top_k, seconds)
for k in range(1, 11):
    start_time = time.time()
    sequences = pipeline(
        'I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?\n',
        do_sample=True,
        top_k=k,  # top_k=10 takes 10 seconds with do_sample=True
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=200,
    )
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"top_K: {k}, end-to-end inference time: {execution_time}s")

/opt/conda/envs/py39_llama_hf/lib/python3.9/site-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


top_K: 1, end-to-end inference time: 1.7114698886871338s
top_K: 2, end-to-end inference time: 10.269404411315918s
top_K: 3, end-to-end inference time: 10.34311842918396s
top_K: 4, end-to-end inference time: 10.357712030410767s
top_K: 5, end-to-end inference time: 0.26271843910217285s
top_K: 6, end-to-end inference time: 10.357895135879517s
top_K: 7, end-to-end inference time: 4.958071947097778s
top_K: 8, end-to-end inference time: 10.36292815208435s
top_K: 9, end-to-end inference time: 2.7058143615722656s
top_K: 10, end-to-end inference time: 1.033567190170288s
